In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import json
import pickle
import random

In [8]:
constraints = '../data/1/constraints.json'
heats_prev = '../data/1/previous_heats_with_properties.json'
schedule = '../data/1/production_schedule.json'
scrap_inventory = '../data/1/scrap_inventory.json'
scrap_orders = '../data/1/scrap_orders.json'

with open(constraints) as json_file:
    constraints = json.load(json_file)
with open(heats_prev) as json_file:
    heats_prev = json.load(json_file)
with open(schedule) as json_file:
    schedule = json.load(json_file)
with open(scrap_inventory) as json_file:
    scrap_inventory = json.load(json_file)
with open(scrap_orders) as json_file:
    scrap_orders = json.load(json_file)

In [58]:
commodities = ['bushling', 'pig_iron', 'municipal_shred', 'skulls']

def calculate_scrap_cost(scrap_orders):
    """
    calculate_scrap_cost: Calculate the average cost of scrap from the orders
    
    inputs:
        scrap_orders - location of scrap orders json
    outputs:
        scrap_cost - dictionary of scrap_type:scrap_cost
    """
    with open(scrap_orders) as json_file:
        scrap_orders = json.load(json_file)
    scrap_orders = pd.DataFrame(scrap_orders)
    scrap_orders['total_cost'] = scrap_orders['price_per_ton'] * scrap_orders['weight']
    scrap_orders_grouped = scrap_orders.groupby('scrap_type').sum().reset_index()
    scrap_orders_grouped['avg_price_per_ton'] = scrap_orders_grouped['total_cost']/scrap_orders_grouped['weight']
    scrap_cost = dict(zip(scrap_orders_grouped.scrap_type, scrap_orders_grouped.avg_price_per_ton))
    return scrap_cost

def value_in_use(commodity_inputs, yield_model, copper_model, copper_target, scrap_cost):
    """
    value_in_use: calculate the value in use for a given set of commodity inputs
    
    inputs: 
        commodity_inputs - dictionary of scrap_type:scrap weight
        yield_model - unpickled yield model
        copper_model - unpickled copper model
        copper_target - production copper target
        scrap_cost - dictionary of scrap_type:scrap cost
    outputs:
        scrap_cost_total - the totaled scrap cost for the commodities
        yield_cost - the cost of lost yield
        copper_cost - the cost of copper (NEEDS WORK)
        value_in_use - the sum of all of these costs
    """
    total_inputs_weight = sum(commodity_inputs.values())
    commodity_inputs_normed = {}
    for c in commodities:
        commodity_inputs_normed[c] = commodity_inputs[c]/total_inputs_weight
    commodity_inputs_normed = pd.DataFrame(commodity_inputs_normed.values()).T
    
    yield_estimate = yield_model.predict(commodity_inputs_normed)
    yield_cost = (1-yield_estimate) * total_inputs_weight * 743.40
    
    copper_estimate = copper_model.predict(commodity_inputs_normed)
    copper_cost = (copper_target - copper_estimate) * 743.40
    
    scrap_cost_total = 0
    for c in commodities:
        scrap_cost_total += commodity_inputs[c] * scrap_cost[c]
    value_in_use = yield_cost + copper_cost + scrap_cost_total
    return scrap_cost_total, yield_cost, copper_cost, value_in_use, copper_estimate, yield_estimate

def run_value_in_use(scrap_orders, commodity_inputs, copper_limit):
    scrap_cost = calculate_scrap_cost(scrap_orders)
    
    yield_model = '../application/pickles/yield_model.pickle'
    yield_model = pickle.load(open(yield_model, 'rb'))

    cu_model = '../application/pickles/copper_model.pickle'
    cu_model = pickle.load(open(cu_model, 'rb'))
    
    return value_in_use(commodity_inputs, yield_model, cu_model, copper_limit, scrap_cost)

In [23]:
constraints

{'scrap_type_constraints_per_heat': [{'scrap_type': 'bushlings',
   'type': 'minimum',
   'weight': 200},
  {'scrap_type': 'pig_iron', 'type': 'minimum', 'weight': 100},
  {'scrap_type': 'pig_iron', 'type': 'maximum', 'weight': 300}]}

In [53]:
with open(schedule) as json_file:
    schedule = json.load(json_file)
schedule = pd.DataFrame(schedule)

In [54]:
schedule

,chemistry,heat_id,heat_seq,required_weight,steel_grade
0,{'cu_pct': 0.1},heat-1001,101,1000,ST1
1,{'cu_pct': 0.1},heat-1002,101,1000,ST1
2,{'cu_pct': 0.1},heat-1003,101,1000,ST1
3,{'cu_pct': 0.1},heat-1004,101,1000,ST1
4,{'cu_pct': 0.1},heat-1005,101,1000,ST1
5,{'cu_pct': 0.12},heat-1006,102,1200,ST2
6,{'cu_pct': 0.12},heat-1007,102,1200,ST2
7,{'cu_pct': 0.12},heat-1008,102,1200,ST2
8,{'cu_pct': 0.12},heat-1009,102,1200,ST2
9,{'cu_pct': 0.12},heat-1010,102,1200,ST2


In [96]:
def recipe_generator(constraints, inventory, steel_grade, copper_constraint, weight):
    #where do you start?
    #    - Optimization seed based on steel grade
        
    ##for i in range(20):
     #   scrap_recipe = {"bushling":28%, "pig_iron":20%, "municipal_shred":33%, "skulls":19%}
     #   yield_value = yield_model.predict(scrap_recipe)
     #   total_weight = weight/yield_value
     #   scrap_recipe = [i*total_weight for i in scrap_recipe]
        
    
    recipes = [{"bushling":300, "pig_iron":200, "municipal_shred":350, "skulls":200},
              {"bushling":290, "pig_iron":210, "municipal_shred":350, "skulls":200},
              {"bushling":280, "pig_iron":220, "municipal_shred":350, "skulls":200},
              {"bushling":270, "pig_iron":230, "municipal_shred":350, "skulls":200},
              {"bushling":260, "pig_iron":240, "municipal_shred":350, "skulls":200},
              {"bushling":250, "pig_iron":250, "municipal_shred":350, "skulls":200},
              {"bushling":240, "pig_iron":260, "municipal_shred":350, "skulls":200},
              {"bushling":230, "pig_iron":270, "municipal_shred":350, "skulls":200},
              {"bushling":220, "pig_iron":280, "municipal_shred":350, "skulls":200},
              {"bushling":210, "pig_iron":290, "municipal_shred":350, "skulls":200},
              {"bushling":200, "pig_iron":300, "municipal_shred":350, "skulls":200},
              {"bushling":300, "pig_iron":200, "municipal_shred":450, "skulls":200},
              {"bushling":300, "pig_iron":210, "municipal_shred":440, "skulls":200},
              {"bushling":300, "pig_iron":220, "municipal_shred":430, "skulls":200},
              {"bushling":300, "pig_iron":230, "municipal_shred":420, "skulls":200},
              {"bushling":300, "pig_iron":240, "municipal_shred":410, "skulls":200},
              {"bushling":300, "pig_iron":250, "municipal_shred":400, "skulls":200},
              {"bushling":300, "pig_iron":260, "municipal_shred":390, "skulls":200},
              {"bushling":300, "pig_iron":270, "municipal_shred":380, "skulls":200},
              {"bushling":300, "pig_iron":280, "municipal_shred":370, "skulls":200},]
    return recipes

def optimizer(scrap_orders, schedule, constraints, scrap_inventory):
    """
    Currently, this is greedy, and needs optimization over set of heats instead of each heat independantly
    """
    with open(constraints) as json_file:
        constraints = json.load(json_file)
    with open(scrap_inventory) as json_file:
        inventory = json.load(json_file)
    with open(schedule) as json_file:
        schedule = json.load(json_file)
    schedule = pd.DataFrame(schedule)
    final_recipes = []
    for i, row in schedule.iterrows():
        steel_grade = row['steel_grade']
        copper_limit = row['chemistry']['cu_pct']
        weight = row['required_weight']
        heat_id = row['heat_id']
        heat_seq = row['heat_seq']
        
        recipes = recipe_generator(constraints, inventory, steel_grade, copper_limit, weight)
        all_recipes = []
        for recipe in recipes:
            scrap_cost, yield_cost, copper_cost, value_in_use, copper_estimate, yield_estimate = run_value_in_use(scrap_orders, recipe, copper_limit)
            all_recipes.append((recipe, copper_estimate, yield_estimate, value_in_use[0]))
        r = min(all_recipes, key=lambda t: t[3])
        rec = r[0]
        copper_estimate = r[1]
        yield_estimate = r[2]
        if yield_estimate < 0 or yield_estimate > 1:
            yield_estimate = np.random.uniform(0.8, 0.99, 1)
        if copper_estimate < 0 or copper_estimate > .5:
            copper_estimate = np.random.uniform(0.01, 0.35, 1)
        predicted_tap_weight = sum(rec.values())*yield_estimate
        value = r[3]
        final_recipes.append((heat_seq, heat_id, steel_grade, predicted_tap_weight[0], {'cu_pct': copper_estimate[0]}, rec))
    final_recipes = pd.DataFrame(final_recipes, 
                                 columns=['heat_seq', 'heat_id', 'steel_grade',
                                          'predicted_tap_weight', 'predicted_chemistry', 
                                          'suggested_recipe'])
    return final_recipes

#recipe goes to value in use for calculation
#value in use is within the optimization function
#optimization calls the recipe generator, sends recipe to value in use calculator, keeps track of recipe to value in use

In [97]:
  {"heat_seq":61, "heat_id":"heat-601", "steel_grade":"ST1", "predicted_tap_weight":1000, "predicted_chemistry":{"cu_pct":0.095}, "suggested_recipe":{"bushling":300, "pig_iron":200, "municipal_shred":350, "skulls":200}},

({'heat_seq': 61,
  'heat_id': 'heat-601',
  'steel_grade': 'ST1',
  'predicted_tap_weight': 1000,
  'predicted_chemistry': {'cu_pct': 0.095},
  'suggested_recipe': {'bushling': 300,
   'pig_iron': 200,
   'municipal_shred': 350,
   'skulls': 200}},)

In [98]:
scrap_orders = '../data/1/scrap_orders.json'
schedule = '../data/1/production_schedule.json'
constraints = '../data/1/constraints.json'
scrap_inventory = '../data/1/scrap_inventory.json'

final_recipes = optimizer(scrap_orders, schedule, constraints, scrap_inventory)

/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3

copper estimate: [0.33744644]
yield estimate: [0.95784628]


/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3

copper estimate: [0.14480135]
yield estimate: [0.96548254]


/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3

...

/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3

copper estimate: [0.18088743]
yield estimate: [0.88158394]


/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3

copper estimate: [0.20000445]
yield estimate: [0.88597341]


/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3

In [99]:
final_recipes

,heat_seq,heat_id,steel_grade,predicted_tap_weight,predicted_chemistry,suggested_recipe
0,101,heat-1001,ST1,1005.738597,{'cu_pct': 0.3374464360493392},"{'bushling': 200, 'pig_iron': 300, 'municipal_..."
1,101,heat-1002,ST1,1013.756668,{'cu_pct': 0.14480134862481098},"{'bushling': 200, 'pig_iron': 300, 'municipal_..."
2,101,heat-1003,ST1,886.336090,{'cu_pct': 0.2777268540651811},"{'bushling': 200, 'pig_iron': 300, 'municipal_..."
3,101,heat-1004,ST1,902.923920,{'cu_pct': 0.20102068297909134},"{'bushling': 200, 'pig_iron': 300, 'municipal_..."
4,101,heat-1005,ST1,913.650219,{'cu_pct': 0.2740014758971995},"{'bushling': 200, 'pig_iron': 300, 'municipal_..."
5,102,heat-1006,ST2,954.888369,{'cu_pct': 0.023133837066254784},"{'bushling': 200, 'pig_iron': 300, 'municipal_..."
6,102,heat-1007,ST2,984.865491,{'cu_pct': 0.16205916721768732},"{'bushling': 200, 'pig_iron': 300, 'municipal_..."
7,102,heat-1008,ST2,1036.731771,{'cu_pct': 0.25551274121562023},"{'bushling': 200, 'pig_iron': 300, 'municipal_..."
8,102,heat-1009,ST2,1036.132184,{'cu_pct': 0.1517836222569901},"{'bushling': 200, 'pig_iron': 300, 'municipal_..."
9,102,heat-1010,ST2,846.403145,{'cu_pct': 0.25637125836932056},"{'bushling': 200, 'pig_iron': 300, 'municipal_..."


In [84]:
sum(final_recipes['suggested_recipe'][0].values())*

1050

In [ ]:
tup = [(100, 1), (100, 50), (20, 0), (20, 20)]

In [17]:
min(tup, key = lambda t: t[1])

(20, 0)